In [ ]:
import sys
sys.path.append("../src")
import pandas as pd



In [ ]:
from vnstock import Listing
list = Listing(source='vci')
VN30 = list.symbols_by_group('VN30').tolist()

In [ ]:
tickers = [t for t in VN30]
#tickers = ['ACB', 'VIC', 'HPG']
day_range = 520


In [ ]:
from fetch_info import fetch_stock
panel = fetch_stock(tickers, hist=day_range, combine = False)
panel

In [ ]:
parent = []
for t in panel:
    stock = panel[t]
    stock['ret'] = (stock['close'].pct_change())
    stock['ticker'] = t
    parent.append(stock[['time','ticker', 'close', 'ret']])

sharpe_panel = pd.concat(parent)
sharpe_panel

In [ ]:
summary_panel = sharpe_panel.groupby('ticker')['ret'].agg(['mean','std', 'var'])
summary_panel['sharpe'] = summary_panel['mean']/summary_panel['std']
summary_panel.sort_values('sharpe', ascending=False)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(summary_panel["mean"], summary_panel["var"])

for x, y, label in zip(summary_panel["mean"], summary_panel["var"], summary_panel.index):
    plt.annotate(label, (x, y), xytext=(4, 4), textcoords="offset points", fontsize=8)

plt.xlabel("mean")
plt.ylabel("var")
plt.show()

In [ ]:
import itertools

combos = itertools.combinations(VN30, 3)

for combo in combos:
    print(combo)

In [ ]:
from mpt import mpt_sim 

final = mpt_sim(sharpe_panel, 100000)



In [ ]:
pd.DataFrame(final)
final

In [ ]:
# Highest Return and Lowest Volatility
best = final.sort_values(['Return','Volatility'],ascending=[False,True]).head(5000)

plt.scatter(best['Volatility']*day_range,best['Return']*day_range, colorizer=(best['Return']/best['Volatility']))

In [ ]:
best = final.sort_values(['Return','Volatility'],ascending=[False,True]).head(100)
fin = best.drop(['Return', "Volatility"], axis = 1)
fin = fin.agg('mean')
fin.sort_values(ascending=False)